In [1]:
import requests
from scripts.utils.helper import *
from scripts.utils.config import dbInfor
import numpy as np
import pandas as pd
import pymysql
import sqlalchemy
import sys
from datetime import datetime, timedelta

con = get_mysql_connection()
query_ticker = "select distinct ticker from ticker_overview"
list_ticker = get_data_from_mysql(con, query_ticker)

In [2]:
## get date of x session before
def get_day_session_before(con, day, x):
    query = f"SELECT date FROM price_board WHERE ticker = 'FPT' AND date < '{day}' ORDER BY date DESC LIMIT {x}, 1"
    date = get_data_from_mysql(con, query)
    return str(date.iloc[0]['date'].date())

In [4]:
day = str(datetime.strptime("2022-02-23", "%Y-%m-%d").date())

In [4]:
day = str((datetime.today() - timedelta(days=1)).date())

In [5]:
day_65 = get_day_session_before(con, day, 65)
day_130 = get_day_session_before(con, day, 130)
day_260 = get_day_session_before(con, day, 260)

In [6]:
query = f"""
    SELECT ticker,
        max(CASE WHEN date = '{day}' THEN close END) as c,
        max(CASE WHEN date = '{day_65}' THEN close END) AS c_65,
        max(CASE WHEN date = '{day_130}' THEN close END) AS c_130,
        max(CASE WHEN date = '{day_260}' THEN close END) AS c_260
    FROM price_board
    where date = '{day}' or date = '{day_65}' or date = '{day_130}' or date = '{day_260}'
    group by ticker
    """
rsidb = get_data_from_mysql(con, query)

In [7]:
list_ticker = get_data_from_mysql(con,"""select distinct t1.ticker from (price_board as t1 join (select ticker from ticker_overview where comGroupCode = "VNINDEX") as t2 on t1.ticker = t2.ticker)""")

In [8]:
rsidb = rsidb.dropna(subset=['c', 'c_65', 'c_130', 'c_260'])

In [9]:
rsidb['roc_65'] = (rsidb['c'] - rsidb['c_65']) / rsidb['c_65']
rsidb['roc_130'] = (rsidb['c'] - rsidb['c_130']) / rsidb['c_130']
rsidb['roc_260'] = (rsidb['c'] - rsidb['c_260']) / rsidb['c_260']
rsidb['score'] = 0.4 * rsidb['roc_65'] + 0.3 * rsidb['roc_130'] + 0.3 * rsidb['roc_260']

In [10]:
df_all = rsidb.merge(list_ticker, on='ticker')

In [11]:
df_all['rank'] = df_all['score'].rank()

In [12]:
n = len(df_all)

In [13]:
df_all['rank'] = df_all['rank'].apply(lambda x: int((x-1)/n * 100))

In [15]:
df_all[df_all['ticker'] == 'FPT']

,ticker,c,c_65,c_130,c_260,roc_65,roc_130,roc_260,score,rank
114,FPT,75916.0,79426.0,75765.0,44146.0,-0.044192,0.001993,0.719658,0.198818,41


In [16]:
eps_growth_query = """
    select t1.ticker, (eps - pre_eps) / pre_eps * 100 as growth_eps from
(select ticker, max(year) as year, max(quarter) as quarter, earningPerShare as eps
from financial_ratio
where quarter < 5
group by ticker) as t1
join 
(select ticker,year, quarter, earningPerShare as pre_eps
from financial_ratio
) as t2
on t2.year = t1.year -1 and t1.quarter = t2.quarter and t1.ticker = t2.ticker
"""

In [17]:
eps_growth = get_data_from_mysql(con, eps_growth_query)

In [18]:
df_all = df_all.merge(eps_growth[eps_growth['growth_eps'] >= 10], on='ticker')

In [19]:
profit_growth_query = """
select t1.ticker, (profit - pre_profit) / pre_profit * 100 as growth_profit from
(select ticker, max(year) as year, postTaxProfit as profit
from income_statement
where quarter = 5
group by ticker) as t1
join
(select ticker, year, postTaxProfit as pre_profit
from income_statement
where quarter = 5) as t2
on t2.year = t1.year - 3 and t1.ticker = t2.ticker
"""

In [20]:
profit_growth = get_data_from_mysql(con, profit_growth_query)

In [21]:
df_all = df_all.merge(profit_growth[profit_growth['growth_profit'] > 10], on='ticker')

In [22]:
roe_query = """
select ticker, max(year) as year, max(quarter) as quarter, roe
from financial_ratio
group by ticker
"""
roe = get_data_from_mysql(con, roe_query)

In [23]:
df_all = df_all.merge(roe[roe['roe'] >= 0.10], on='ticker')

In [25]:
pre_month = str((datetime.strptime("2022-02-23", "%Y-%m-%d") - timedelta(days=30)).date())

In [31]:
avg_volume_query = f"""
select ticker, avg(volume) as volume from price_board
where date > '{pre_month}' and date <= '{day}'
group by ticker
"""
avg_volume = get_data_from_mysql(con, avg_volume_query)

In [33]:
df_all = df_all.merge(avg_volume[avg_volume['volume'] > 1000], on='ticker')

In [34]:
df_all

,ticker,c,c_65,c_130,c_260,roc_65,roc_130,roc_260,score,rank,growth_eps,growth_profit,year,quarter,roe,volume
0,HAG,11250.0,8150.0,5250.0,4500.0,0.380368,1.142857,1.500000,0.945004,88,389.898990,1266.6667,2022,5,0.196,18531564.7059
1,HAS,11920.0,12264.0,12068.0,8147.0,-0.028050,-0.012264,0.463115,0.124036,25,301.604278,60.0000,2022,5,0.127,1729.4118
2,ITA,16700.0,11350.0,6510.0,5980.0,0.471366,1.565284,1.792642,1.195924,94,29.677419,32.5301,2022,5,0.196,9670076.4706
3,ITD,15550.0,20894.0,17099.0,9375.0,-0.255767,-0.090590,0.658667,0.068116,17,78.382148,30.2326,2022,5,0.284,137664.7059
4,QCG,14100.0,10250.0,7000.0,7170.0,0.375610,1.014286,0.966527,0.744488,81,31.914894,27.7228,2022,5,0.180,1229488.2353
5,SFG,18521.0,16299.0,11903.0,7162.0,0.136327,0.555994,1.586009,0.697132,79,115.833333,950.0000,2022,5,0.149,36505.8824
6,SJS,85100.0,68100.0,64900.0,32700.0,0.249633,0.311248,1.602446,0.673962,78,300.936768,52.6316,2022,5,0.360,35452.9412
7,SPM,18607.0,20251.0,12968.0,12310.0,-0.081181,0.434840,0.511535,0.251440,49,483.754266,122.2222,2022,5,0.519,4058.8235
8,TCR,7500.0,6500.0,3880.0,3860.0,0.153846,0.932990,0.943005,0.624337,76,460.679612,637.5000,2022,5,0.132,18300.0000
9,TRC,45451.0,44483.0,33568.0,32731.0,0.021761,0.353998,0.388622,0.231491,47,106.334099,14.4578,2022,5,0.427,3305.8824


In [ ]:
con.close()